# Preliminary direct numerical simulation of 2D Rayleigh-Bénard convection
In this notebook, I present the first Dedalus simulations and examine their qualitative features.

## Theoretical formulation
I consider the nondimensionalised equations in the Boussinesq approximation given by [Busse (1978)](https://doi.org/10.1088/0034-4885/41/12/003):
$$
\begin{align*}
    \mathrm{Pr}^{-1} \left[\partial_t \vec{u} + (\vec{u} \cdot \nabla) \vec{u}\right] &= - \nabla \pi + \nabla^2 \vec{u} + \theta \hat{z}, \\
    \partial_t \theta + (\vec{u} \cdot \nabla) \theta &= \mathrm{Ra}\, (\vec{u} \cdot \hat{z}) + \nabla^2 \theta, \\
    \nabla \cdot \vec{u} &= 0
\end{align*}
$$
where:
- $\vec{u} = (u,0,w)$ is the dimensionless velocity,
- $\theta$ is the dimensionless temperature perturbation from the conductive (linear) base profile,
- $\pi$ is the dimensionless pressure perturbation from the hydrostatic base profile,
- $\mathrm{Pr}$ is the Prandtl number, the ratio of kinematic viscosity to thermal diffisivity,
- $\mathrm{Ra}$ is the Rayleigh number, the ratio of the thermal diffusion time scale to the convective time scale.

The equations have been nondimensionalised using:
- The layer thickness $d$ as the length scale,
- The thermal diffusion time $d^2/D$, where $D$ is the thermal diffusivity, as the time scale, and
- A temperature perturbation scale $\delta T/\mathrm{Ra}$, where $\delta T$ is the temperature difference between the top and bottom plates.

I impose no-slip, isothermal boundary conditions at the top and bottom of the domain,
$$
\begin{alignat*}{2}
    \vec{u}(z=0) &= \vec{u}(z=L_z) &&= \vec{0}, \\
    \theta(z=0) &= \theta(z=L_z) &&= 0,
\end{alignat*}
$$
and periodic lateral boundary conditions
$$
\begin{align*}
    \vec{u}(x=0) &= \vec{u}(x=L_x),\\
    \theta(x=0) &= \theta(x=L_x).
\end{align*}
$$

### Tau terms
Dedalus requires us to add *tau terms* of the form $\tau(x) P(z)$, where $\tau$ is a new degree of freedom and $P$ is a known polynomial, to the equations. This allows the non-periodic top and bottom boundary conditions to be imposed while ensuring that the system still has exact polynomial solutions (i.e., so it is solvable using spectral methods). Following [the steps in the documentation](https://dedalus-project.readthedocs.io/en/latest/pages/tau_method.html), I introduce six new degrees of freedom
$$
    \vec{\tau}_{u1}(x) = (\tau_1(x), 0, \tau_2(x)), \quad \vec{\tau}_{u2}(x) = (\tau_3(x), 0, \tau_4(x)),
    \quad \tau_{\theta 1}(x), \quad \text{and } \tau_{\theta 2}(x).
$$
Defining the auxiliary variables
$$
\begin{align*}
    G_u &= \nabla \otimes \vec{u} - \hat{z} \otimes \vec{\tau}_{u1}(x) P(z), \\
    \vec{G}_\theta &= \nabla \theta - \hat{z} \tau_{\theta 1}(x) P(z),
\end{align*}
$$
the equations become
$$
\begin{align*}
    \mathrm{Pr}^{-1} \partial_t \vec{u} - \nabla \cdot G_u + \nabla \pi - \theta \hat{z} + \vec{\tau}_{u2}(x) P(z) &= -\mathrm{Pr}^{-1} \vec{u} \cdot G_u, \\
    \partial_t \theta - \nabla \cdot \vec{G}_\theta - \mathrm{Ra}\, (\vec{u} \cdot \hat{z}) + \tau_{\theta 2}(x) P(z) &= -\vec{u} \cdot \vec{G}_\theta, \\
    \operatorname{Tr}(G_u) &= 0.
\end{align*}
$$

### Pressure gauge condition
Since only the gradient of $\pi$ appears in the equations, there is a gauge freedom in the problem: the pressure is only determined up to a constant. Dedalus requires us to explicitly specify gauge conditions, so since $\pi$ is a pressure perturbation, I require its mean value to be zero:
$$\int \pi ~\mathrm{d}V = 0.$$
Following [the documentation](https://dedalus-project.readthedocs.io/en/latest/pages/gauge_conditions.html), I must also introduce a seventh spatially constant tau variable $\tau_\pi$ to the divergence equation:
$$\operatorname{Tr}(G_u) + \tau_\pi = 0.$$

### Final PDE system
The final system of equations that I have entered into Dedalus is
$$
\begin{align*}
    \mathrm{Pr}^{-1} \partial_t \vec{u} - \nabla \cdot G_u + \nabla \pi - \theta \hat{z} + \vec{\tau}_{u2}(x) P(z) &= -\mathrm{Pr}^{-1} \vec{u} \cdot G_u, \\
    \partial_t \theta - \nabla \cdot \vec{G}_\theta - \mathrm{Ra}\, (\vec{u} \cdot \hat{z}) + \tau_{\theta 2}(x) P(z) &= -\vec{u} \cdot \vec{G}_\theta, \\
    \operatorname{Tr}(G_u) + \tau_\pi = 0,
\end{align*}
$$
with the substitutions
$$
\begin{align*}
    G_u &= \nabla \otimes \vec{u} - \hat{z} \otimes \vec{\tau}_{u1}(x) P(z), \\
    \vec{G}_\theta &= \nabla \theta - \hat{z} \tau_{\theta 1}(x) P(z),
\end{align*}
$$
subject to the boundary conditions
$$
\begin{alignat*}{2}
    \vec{u}(z=0) &= \vec{u}(z=L_z) &&= \vec{0}, \\
    \theta(z=0) &= \theta(z=L_z) &&= 0, \\
    \int \pi ~\mathrm{d}V &= 0. && \\
\end{alignat*}
$$
The periodic lateral boundaries are handled automatically by using a Fourier basis in the horizontal.

## Implementation and analysis
I have adapted the code from [this example notebook](https://dedalus-project.readthedocs.io/en/latest/pages/examples/ivp_2d_rayleigh_benard.html) for 2D RBC in the Dedalus documentation.

The choice of parameters is informed by the scales used to nondimensionalise the equations:
- The dimensionless length unit is the layer thickness $d$, so the height of the domain must be 1 and the width should be greater than 1 but still on the order of unity,
- The time unit is the thermal diffusion time, which is $\mathrm{Ra}$ times larger than the convective time scale, so in order to resolve convective motions the time step should be on the order of $1/\mathrm{Ra}$,
- Convective instability only occurs for sufficiently large Rayleigh numbers ($\gtrsim 10^3$ for most cases in the literature).

The fluid is initially at rest, with a random, normally distributed temperature perturbation at each point. Since the dimensionless temperature unit is $\delta T/\mathrm{Ra}$ and one expects the solution to have temperature variations on the order of $\delta T$, the initial perturbation should be a few orders of magnitude smaller than $\mathrm{Ra}$.

In [100]:
from IPython.display import Video

### Fine resolution run #1
This run has:
- Aspect ratio 4
- $\mathrm{Ra} = 10^6$
- $\mathrm{Pr} = 1$
- 256 horizontal modes
- 64 vertical modes
- Time step $1/\mathrm{Ra} = 10^{-6}$
- Simulation length $10^{-1}$ time units

Observe that while the flow is initially unsteady, the transient behaviour diminishes and the system converges to a stable periodic solution with four convective cells after about 0.03 time units.

In [99]:
Video('../data/prelim_dns_fine/prelim_dns_fine_s1.mp4')

### Fine resolution run #2
I now increase the Rayleigh number by a factor of 10 while keeping the same time step:
- Aspect ratio 4
- $\mathrm{Ra} = 10^7$
- $\mathrm{Pr} = 1$
- 256 horizontal modes
- 64 vertical modes
- Time step $10/\mathrm{Ra} = 10^{-6}$
- Simulation length $10^{-1}$ time units

The flow now reaches a slightly less steady state.

In [132]:
Video('../data/prelim_dns_fine2/prelim_dns_fine2_s1.mp4')

### Fine resolution run #3
I now increase the Rayleigh number by another factor of 10:
- Aspect ratio 4
- $\mathrm{Ra} = 10^8$
- $\mathrm{Pr} = 1$
- 256 horizontal modes
- 64 vertical modes
- Time step $10/\mathrm{Ra} = 10^{-7}$
- Simulation length $10^{-2}$ time units

The system is now beginning to exhibit the desired properties. After the initial transients have disappeared, there is still turbulent fine-scale motion superimposed on the large-scale circulation.

In [135]:
Video('../data/prelim_dns_fine3/prelim_dns_fine3_s1.mp4')